# Import packages

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Load data

In [2]:
train_df = pd.read_csv('/kaggle/input/song-popularity-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/song-popularity-prediction/test.csv')
print(train_df.info())
print('---------------------')
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                40000 non-null  int64  
 1   song_duration_ms  35899 non-null  float64
 2   acousticness      36008 non-null  float64
 3   danceability      35974 non-null  float64
 4   energy            36025 non-null  float64
 5   instrumentalness  36015 non-null  float64
 6   key               35935 non-null  float64
 7   liveness          35914 non-null  float64
 8   loudness          36043 non-null  float64
 9   audio_mode        40000 non-null  int64  
 10  speechiness       40000 non-null  float64
 11  tempo             40000 non-null  float64
 12  time_signature    40000 non-null  int64  
 13  audio_valence     40000 non-null  float64
 14  song_popularity   40000 non-null  int64  
dtypes: float64(11), int64(4)
memory usage: 4.6 MB
None
---------------------
<class 'pandas

In [3]:
column_names = train_df.columns
print(f"[BEFORE] Number of NANs in the data frame {train_df.isna().sum().sum()}")
imp = IterativeImputer(random_state=42, max_iter=50, initial_strategy='mean')
train_df = pd.DataFrame(imp.fit_transform(train_df), columns = column_names)
print(f"[AFTER] Number of NANs in the data frame {train_df.isna().sum().sum()}")

[BEFORE] Number of NANs in the data frame 32187
[AFTER] Number of NANs in the data frame 0


In [4]:
train_df.describe()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
count,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,19999.50000,193164.915771,0.276022,0.570949,0.683732,0.036651,5.043749,0.198503,-7.407356,0.321150,0.094107,116.562815,3.394375,0.580645,0.364400
std,11547.14972,43532.153587,0.288565,0.182279,0.207593,0.142783,3.209573,0.143999,3.781104,0.466924,0.083591,26.167911,0.524405,0.237351,0.481268
min,0.00000,25658.000000,-0.103450,0.043961,-0.040054,-0.030025,0.000000,0.027843,-32.117911,0.000000,0.015065,62.055779,2.000000,0.013398,0.000000
25%,9999.75000,168444.500000,0.045912,0.440183,0.546396,0.001015,2.000000,0.114203,-9.494627,0.000000,0.038500,96.995309,3.000000,0.398669,0.000000
50%,19999.50000,188540.500000,0.152236,0.600681,0.707325,0.002158,5.243190,0.147288,-6.410053,0.000000,0.055881,113.795959,3.000000,0.598827,0.000000
75%,29999.25000,211122.500000,0.462658,0.708747,0.861742,0.003790,8.000000,0.216083,-4.687936,1.000000,0.118842,128.517383,4.000000,0.759635,1.000000
max,39999.00000,491671.000000,1.065284,0.957131,1.039741,1.075415,11.000000,1.065298,-0.877346,1.000000,0.560748,219.163578,5.000000,1.022558,1.000000


In [5]:
from sklearn.model_selection import train_test_split

X = train_df.drop(['id', 'song_popularity'], axis=1)
y = train_df['song_popularity']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

In [6]:
from sklearn.ensemble import ExtraTreesRegressor
reg = ExtraTreesRegressor(n_estimators=500, random_state=0).fit(
   X_train, y_train)
reg.score(X_test, y_test)

0.0017182557849287905

In [7]:
column_names = test_df.columns
imp = IterativeImputer(random_state=42, max_iter=50, initial_strategy='mean')
test_df = pd.DataFrame(imp.fit_transform(test_df), columns = column_names)

X_pred = test_df.drop(['id'], axis=1)
y_pred = reg.predict(X_pred)

test_df['song_popularity'] = y_pred
submission = test_df[['id', 'song_popularity']]
submission.to_csv('submission.csv', index=False)